#### Restart and Run All

In [1]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
#data_path = "\\users\\user\\onedrive\\a7\\Data"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
today

datetime.date(2023, 8, 25)

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2023-08-25
yesterday: 2023-08-24


In [3]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

### Begin of Tables in the process

In [4]:
sql = """
SELECT * 
FROM sales 
ORDER BY id DESC LIMIT 1
"""
tmp = pd.read_sql(sql, conlite)
tmp["fm_date"] = pd.to_datetime(tmp["fm_date"])
tmp["to_date"] = pd.to_datetime(tmp["to_date"])
tmp["created_at"] = pd.to_datetime(tmp["created_at"])
tmp["updated_at"] = pd.to_datetime(tmp["updated_at"])
tmp.style.format(format_dict)

,id,name,fm_date,to_date,days,fm_price,to_price,diff,pct,ttl_spread,avg_spread,max_price,min_price,qty,created_at,updated_at,latest_date_id
0,1302998,WHAIR,2023-08-24,2023-08-24,0,6.75,6.90,0.15,2.22%,3,0,7.05,6.75,"3,487,364",2023-08-24,2023-08-24,1


In [5]:
names = tmp["name"]
type(names)

pandas.core.series.Series

In [6]:
name = names.to_string(index=False)
type(name)

str

In [7]:
sql = """
SELECT * 
FROM stocks
WHERE name = '%s'
"""
sql = sql % name
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.style.format(format_dict)


SELECT * 
FROM stocks
WHERE name = 'WHAIR'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,127,WHAIR,0.00,0.00,B,6.70,0.00,0.00,0.00,8.70,"3,000",-4,4,"50,000",0.000000,0.000000,RD25pct,SET


### End of Tables in the process

In [8]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2023-08-24'
ORDER BY a.name



,name
0,AH
1,AIMIRT
2,ASK
3,ASP
4,AWC
5,BBL
6,BCH
7,BEM
8,BH
9,CK


In [9]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2023-08-24' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [10]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AH,2023-08-24,2023-08-24,34.00,34.25,"14,620,540",34.75,32.25,B,SET,0.25,0.74%
1,AWC,2023-08-22,2023-08-24,3.96,4.12,"370,793,768",4.12,3.94,B,SET50,0.16,4.04%
2,DIF,2023-08-15,2023-08-24,9.75,9.15,"672,386,000",9.75,9.15,B,SET,-0.60,-6.15%
3,JASIF,2023-08-24,2023-08-24,6.80,6.85,"28,721,276",6.95,6.75,B,SET,0.05,0.74%
4,JMART,2023-08-24,2023-08-24,21.30,21.20,"217,047,768",21.30,19.10,B,SET50,-0.10,-0.47%
5,JMT,2023-08-24,2023-08-24,45.75,45.50,"66,145,440",45.75,42.25,B,SET50,-0.25,-0.55%
6,MCS,2023-08-24,2023-08-24,6.10,6.05,"1,687,180",6.25,6.05,B,SET,-0.05,-0.82%
7,NER,2023-08-24,2023-08-24,4.56,4.62,"30,309,276",4.72,4.56,B,SET,0.06,1.32%
8,ORI,2023-08-22,2023-08-24,10.10,10.60,"101,521,056",10.60,10.10,B,SET100,0.50,4.95%
9,PTTGC,2023-08-24,2023-08-24,36.00,36.50,"50,133,140",37.00,36.00,B,SET50,0.50,1.39%


In [11]:
df.shape

(48, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [12]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AH,2023-08-24,2023-08-24,34.00,34.25,"14,620,540",34.75,32.25,0.74%,B
1,AWC,2023-08-22,2023-08-24,3.96,4.12,"370,793,768",4.12,3.94,4.04%,B
2,DIF,2023-08-15,2023-08-24,9.75,9.15,"672,386,000",9.75,9.15,-6.15%,B
3,JASIF,2023-08-24,2023-08-24,6.80,6.85,"28,721,276",6.95,6.75,0.74%,B
4,JMART,2023-08-24,2023-08-24,21.30,21.20,"217,047,768",21.30,19.10,-0.47%,B
5,JMT,2023-08-24,2023-08-24,45.75,45.50,"66,145,440",45.75,42.25,-0.55%,B
6,MCS,2023-08-24,2023-08-24,6.10,6.05,"1,687,180",6.25,6.05,-0.82%,B
7,NER,2023-08-24,2023-08-24,4.56,4.62,"30,309,276",4.72,4.56,1.32%,B
8,ORI,2023-08-22,2023-08-24,10.10,10.60,"101,521,056",10.60,10.10,4.95%,B
9,PTTGC,2023-08-24,2023-08-24,36.00,36.50,"50,133,140",37.00,36.00,1.39%,B


In [13]:
file_name = "daily-sales.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(one_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [14]:
df[cols].shape

(48, 10)

In [15]:
sales = df[cols]
sales

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AH,2023-08-24,2023-08-24,34.00,34.25,14620540,34.75,32.25,0.74,B
1,AWC,2023-08-22,2023-08-24,3.96,4.12,370793768,4.12,3.94,4.04,B
2,DIF,2023-08-15,2023-08-24,9.75,9.15,672386000,9.75,9.15,-6.15,B
3,JASIF,2023-08-24,2023-08-24,6.80,6.85,28721276,6.95,6.75,0.74,B
4,JMART,2023-08-24,2023-08-24,21.30,21.20,217047768,21.30,19.10,-0.47,B
5,JMT,2023-08-24,2023-08-24,45.75,45.50,66145440,45.75,42.25,-0.55,B
6,MCS,2023-08-24,2023-08-24,6.10,6.05,1687180,6.25,6.05,-0.82,B
7,NER,2023-08-24,2023-08-24,4.56,4.62,30309276,4.72,4.56,1.32,B
8,ORI,2023-08-22,2023-08-24,10.10,10.60,101521056,10.60,10.10,4.95,B
9,PTTGC,2023-08-24,2023-08-24,36.00,36.50,50133140,37.00,36.00,1.39,B


In [16]:
file_name = "Price-Today.csv"
input_file = data_path + file_name
prices = pd.read_csv(input_file)
prices

,name,date,price,change,%change,open,high,low,volume,value(m)
0,ACE,2023-08-25,1.81,0.01,0.555556,1.81,1.83,1.79,6374347,1.153669e+04
1,ADVANC,2023-08-25,218.00,-2.00,-0.909091,219.00,220.00,216.00,7527632,1.638892e+06
2,AEONTS,2023-08-25,178.50,2.50,1.420455,176.00,179.00,175.50,450511,7.994746e+04
3,AH,2023-08-25,35.00,0.75,2.189781,34.25,35.00,34.00,1188315,4.111882e+04
4,AIMIRT,2023-08-25,11.00,0.10,0.917431,10.90,11.00,10.80,151902,1.655632e+03
...,...,...,...,...,...,...,...,...,...,...
185,WHAIR,2023-08-25,6.95,0.05,0.724638,6.90,7.00,6.85,32302,2.239538e+02
186,WHART,2023-08-25,10.40,0.00,0.000000,10.50,10.60,10.40,987300,1.031226e+04
187,WHAUP,2023-08-25,3.98,0.04,1.015228,3.92,3.98,3.92,1467294,5.810107e+03
188,WICE,2023-08-25,8.15,0.05,0.617284,8.00,8.15,8.00,485562,3.920757e+03


In [17]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,date,price,change,%change,open,high,low,volume,value(m)
0,AH,2023-08-24,2023-08-24,34.00,34.25,14620540,34.75,32.25,0.74,B,2023-08-25,35.00,0.75,2.189781,34.25,35.00,34.00,1188315,4.111882e+04
1,AWC,2023-08-22,2023-08-24,3.96,4.12,370793768,4.12,3.94,4.04,B,2023-08-25,4.10,-0.02,-0.485437,4.12,4.14,4.06,48252126,1.969890e+05
2,DIF,2023-08-15,2023-08-24,9.75,9.15,672386000,9.75,9.15,-6.15,B,2023-08-25,9.30,0.15,1.639344,9.15,9.30,9.15,17514189,1.618335e+05
3,JASIF,2023-08-24,2023-08-24,6.80,6.85,28721276,6.95,6.75,0.74,B,2023-08-25,6.95,0.10,1.459854,6.90,6.95,6.85,6208124,4.288525e+04
4,JMART,2023-08-24,2023-08-24,21.30,21.20,217047768,21.30,19.10,-0.47,B,2023-08-25,22.30,1.10,5.188679,21.30,22.60,21.20,43499248,9.507535e+05
5,JMT,2023-08-24,2023-08-24,45.75,45.50,66145440,45.75,42.25,-0.55,B,2023-08-25,46.50,1.00,2.197802,45.50,46.75,45.50,12636443,5.842312e+05
6,MCS,2023-08-24,2023-08-24,6.10,6.05,1687180,6.25,6.05,-0.82,B,2023-08-25,6.15,0.10,1.652893,6.05,6.20,6.00,260648,1.595076e+03
7,NER,2023-08-24,2023-08-24,4.56,4.62,30309276,4.72,4.56,1.32,B,2023-08-25,4.64,0.02,0.432900,4.60,4.64,4.56,12456468,5.735269e+04
8,ORI,2023-08-22,2023-08-24,10.10,10.60,101521056,10.60,10.10,4.95,B,2023-08-25,10.80,0.20,1.886792,10.70,10.80,10.60,3440524,3.679244e+04
9,PTTGC,2023-08-24,2023-08-24,36.00,36.50,50133140,37.00,36.00,1.39,B,2023-08-25,38.00,1.50,4.109589,36.50,38.00,36.50,24390818,9.132534e+05


In [18]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
colu

['name',
 'fm_date',
 'to_date',
 'fm_price',
 'to_price',
 'qty',
 'max_price',
 'min_price',
 'percent',
 'status',
 'price',
 'change',
 'volume',
 'date']

In [19]:
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AH,2023-08-24,2023-08-24,34.00,34.25,14620540,34.75,32.25,0.74,B,35.00,0.75,1188315,2023-08-25
1,AWC,2023-08-22,2023-08-24,3.96,4.12,370793768,4.12,3.94,4.04,B,4.10,-0.02,48252126,2023-08-25
2,DIF,2023-08-15,2023-08-24,9.75,9.15,672386000,9.75,9.15,-6.15,B,9.30,0.15,17514189,2023-08-25
3,JASIF,2023-08-24,2023-08-24,6.80,6.85,28721276,6.95,6.75,0.74,B,6.95,0.10,6208124,2023-08-25
4,JMART,2023-08-24,2023-08-24,21.30,21.20,217047768,21.30,19.10,-0.47,B,22.30,1.10,43499248,2023-08-25
5,JMT,2023-08-24,2023-08-24,45.75,45.50,66145440,45.75,42.25,-0.55,B,46.50,1.00,12636443,2023-08-25
6,MCS,2023-08-24,2023-08-24,6.10,6.05,1687180,6.25,6.05,-0.82,B,6.15,0.10,260648,2023-08-25
7,NER,2023-08-24,2023-08-24,4.56,4.62,30309276,4.72,4.56,1.32,B,4.64,0.02,12456468,2023-08-25
8,ORI,2023-08-22,2023-08-24,10.10,10.60,101521056,10.60,10.10,4.95,B,10.80,0.20,3440524,2023-08-25
9,PTTGC,2023-08-24,2023-08-24,36.00,36.50,50133140,37.00,36.00,1.39,B,38.00,1.50,24390818,2023-08-25


In [20]:
df_merge[colu].shape

(48, 14)

In [21]:
df_merge.shape

(48, 19)

In [22]:
file_name = "daily-sales-prices.csv"
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(data_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(one_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

In [23]:
### Call ruby ruby\daily-out-new.rb

In [24]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [25]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [26]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AH       2023-08-24 2023-08-25  34.00  35.00   2.94%  14,620,540  35.00  32.25 B Update
AIMIRT   2023-08-25 2023-08-25  10.90  11.00   0.92%  11,656,576  11.20  10.90 I Insert
ASK      2023-08-22 2023-08-25  23.60  25.75   9.11%   1,906,448  25.75  23.60 I Update
ASP      2023-07-27 2023-08-25   2.84   3.00   5.63% 179,542,432   3.00   2.80 I Update
AWC      2023-08-25 2023-08-25   4.12   4.10  -0.49% 370,793,768   4.12   3.94 B Insert
BBL      2023-08-25 2023-08-25 166.50 167.50    0.6% 232,218,352 174.50 166.50 O Insert
BCH      2023-08-22 2023-08-25  18.50  19.40   4.86% 278,319,040  19.40  18.40 I Update
BEM      2023-08-25 2023-08-25   9.10   9.05  -0.55% 401,655,272   9.10   8.70 O Insert
BH       2023-08-25 2023-08-25 260.00 258.00  -0.77%  17,579,323 266.00 245.00 O Insert
CK       2023-08-18 2023-08-25  

In [27]:
os.chdir("C:\\users\\user\\onedrive\\a7\\Daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\Daily'

### cd\ruby\port_lite
### rails runner db/ins_sales.rb